# Clearing Electricity Markets Using PyPSA

In [The Supply and Demand of Electricity](external/ontario_electricity_market/2_the_supply_and_demand_of_electricity.md), we discussed the electricity market and the economic dispatch (ED) optimization problem. In this blog post, we will show how to use PyPSA to recreate the results of [Example 2.4](supply_and_demand_of_electricity_example_2_4). Example 2.4 consists of an electrical grid with two market nodes, $\mathrm{A}$ and $\mathrm{B}$, connected by a lossy transmission line. Node $\mathrm{A}$ has a generator, $\mathrm{G1}$, and node $\mathrm{B}$ has another generator $\mathrm{G2}$ and two loads $\mathrm{L1}$ and $\mathrm{L2}$. The generators and loads have made offers and bids in the electricity market, with price-quantity pairs shown in {ref}`fig_2_3`.

```{figure} img/diagrams-example_2_4-unsolved.png
:label: example_2_4_unsolved
:width: 64%

Network of Example 2.4.
```

In [1]:
import math

We start by defining a PyPSA `Network`:

In [2]:
import pypsa

network = pypsa.Network()

We define the market nodes $\mathrm{A}$ and $\mathrm{B}$ as two electrical [`Bus` components](https://docs.pypsa.org/latest/user-guide/components/buses/).

In [3]:
network.add("Bus", name="A")
network.add("Bus", name="B")

Transmission lines are modeled by the [`Line` component](https://docs.pypsa.org/latest/user-guide/components/lines/), but because the transmission line of Example 2.4 has a fixed efficiency for simplicity, we model it as a [`Link` component](https://docs.pypsa.org/latest/user-guide/components/links/):

<!-- eta = 1 - (R / V_A^2) * Q_G1 => R = (1 - eta) / Q_G1 * V_A^2 = (1 - 0.75) / 6 * 500^2 = 10_416.6666666667 -->

In [4]:
network.add(
    "Link",
    name="line",
    bus0="A",
    bus1="B",
    efficiency=0.75,
    p_nom_extendable=True,
    p_min_pu=-1,
    p_max_pu=1,
)

::: {admonition}
The bidirectional lossy link can also be modeled as two one-way links, as follows:

```python
network.add("Link", name="A-to-B", bus0="A", bus1="B", efficiency=0.75, p_nom=math.inf)
network.add("Link", name="B-to-A", bus0="B", bus1="A", efficiency=0.75, p_nom=math.inf)
```
:::

The generators and loads have piecewise linear dispatch curves, but PyPSA supports only linear or quadratic dispatch curves (using the `marginal_cost` and `marginal_cost_quadratic` parameters). PyPSA does this because it is designed for planning and research purposes. It is designed to work with the typical marginal cost of each generator, rather than the exact price-quantity pairs that each generator has submitted in the past for each market hour.

Following the approach discussed in {ref}`supply_and_demand_of_electricity_higher_dimensional_optimization_problems`, we will model each resource (generator or load) as multiple resources&mdash;one for each price-quantity pair submitted by the resource.

For example, because generator $\mathrm{G1}$ submitted two nonzero price-quantity pairs, we will model the generator in PyPSA as two generators. According to its price-quantity pairs, generator $\mathrm{G1}$ is willing to produce up to $6 \ \mathrm{MW}$ at $\$2/\mathrm{MWh}$, or up to $9 \ \mathrm{MW}$ at $\$7/\mathrm{MWh}$. We model this as a $6\text{-}\mathrm{MW}$ generator that has a marginal cost of $\$2/\mathrm{MWh}$, plus a $3\text{-}\mathrm{MW}$ generator that has a marginal cost of $\$7/\mathrm{MWh}$.

Thus, we define the [`Generator` components](https://docs.pypsa.org/latest/user-guide/components/generators/) as follows:

In [5]:
network.add("Generator", name="G1-1", bus="A", p_nom=6, marginal_cost=2)
network.add("Generator", name="G1-2", bus="A", p_nom=3, marginal_cost=7)

network.add("Generator", name="G2-1", bus="B", p_nom=7, marginal_cost=4)
network.add("Generator", name="G2-2", bus="B", p_nom=3, marginal_cost=10)

We will use the same approach for the loads. PyPSA's [`Load` component](https://docs.pypsa.org/latest/user-guide/components/loads/) does not support the `marginal_cost` parameter, but PyPSA's documentation explicitly states to use the `Generator` component with a negative `sign` and negative `marginal_cost` for this purpose:

In [6]:
network.add("Generator", name="L1-1", bus="B", p_nom=4, marginal_cost=-8, sign=-1)
network.add("Generator", name="L1-2", bus="B", p_nom=4, marginal_cost=-5, sign=-1)

network.add("Generator", name="L2-1", bus="B", p_nom=3, marginal_cost=-9, sign=-1)
network.add("Generator", name="L2-2", bus="B", p_nom=6, marginal_cost=-3, sign=-1)

All of the data we have given are static, but PyPSA is designed to solve an optimization problem spanning multiple snapshots in time. So, we must specify to optimize for an explicit snapshot in time $t_0$:

In [7]:
network.set_snapshots([0])

`Network.optimize()` solves the optimal power flow (OPF) problem, which extends economic dispatch with linearized (i.e., approximate) power flow constraints. In this sense, `optimize()` acts also to simulate the grid and the market clearing process.

In [8]:
network.optimize()

Index(['A', 'B'], dtype='str', name='name')
Index(['line'], dtype='str', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 10 primals, 21 duals
Objective: -4.10e+01
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-ext-p-lower, Link-ext-p-upper were not assigned to the network.


Running HiGHS 1.13.0 (git hash: 1bce6d5): Copyright (c) 2026 under MIT licence terms
LP linopy-problem-6yy3t9h2 has 21 rows; 10 cols; 31 nonzeros
Coefficient ranges:
  Matrix  [8e-01, 1e+00]
  Cost    [2e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [3e+00, 7e+00]
Presolving model
2 rows, 9 cols, 10 nonzeros  0s
1 rows, 8 cols, 8 nonzeros  0s
Dependent equations search running on 1 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
1 rows, 8 cols, 8 nonzeros  0s
Presolve reductions: rows 1(-20); columns 8(-2); nonzeros 8(-23) 
Solving the presolved LP
Using dual simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0.0s
          1    -4.1000000000e+01 Pr: 0(0) 0.0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-6yy3t9h2
Model status        : Optimal
Simplex   iterations: 1
Objective 

('ok', 'optimal')

By inspecting `network.generators_t` and `network.buses_t`, we can see the market-clearing quantities of the generators and loads, as well as the locational marginal prices (LMPs) of the two nodes:

In [9]:
network.generators_t["p"].loc[0].groupby(
    lambda name: name.split("-")[0]
).sum().to_frame(r"$Q^*$ [MW]").rename_axis(None).T

,G1,G2,L1,L2
$Q^*$ [MW],6.0,6.5,8.0,3.0


In [10]:
network.buses_t["marginal_price"].loc[0].to_frame("$P^*$ [$/MWh]").rename_axis("Node").T

Node,A,B
$P^*$ [$/MWh],3.0,4.0
